In [1]:
import os
import datetime
import pandas as pd

# DATA_URL = "https://covid.ourworldindata.org/data/owid-covid-data.csv"

In [2]:
df = pd.read_csv(os.getenv("DATA_URL"))
df.sample(5)

,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million
18128,BEN,Africa,Benin,2020-05-07,140.0,44.0,10.857,2.0,0.0,0.143,...,0.6,12.3,11.035,0.5,61.77,0.545,NaN,NaN,NaN,NaN
125740,OWID_NAM,NaN,North America,2021-10-25,54886984.0,123321.0,85138.857,1113694.0,1934.0,2074.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
83749,IMN,Europe,Isle of Man,2021-05-28,1592.0,1.0,0.143,29.0,0.0,0.000,...,NaN,NaN,NaN,NaN,81.40,NaN,NaN,NaN,NaN,NaN
143711,KNA,North America,Saint Kitts and Nevis,2021-10-20,2589.0,15.0,17.571,21.0,0.0,0.000,...,NaN,NaN,NaN,2.3,76.23,0.779,NaN,NaN,NaN,NaN
175045,TUV,Oceania,Tuvalu,2021-10-11,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,67.57,NaN,NaN,NaN,NaN,NaN


In [3]:
%time df['date'].apply(lambda date: datetime.datetime.strptime(date, '%Y-%m-%d'))
%time pd.to_datetime(df['date'], format='%Y-%m-%d');

CPU times: user 11.1 s, sys: 0 ns, total: 11.1 s
Wall time: 11.1 s
CPU times: user 223 ms, sys: 0 ns, total: 223 ms
Wall time: 224 ms


0        2020-02-24
1        2020-02-25
2        2020-02-26
3        2020-02-27
4        2020-02-28
            ...    
189485   2022-05-22
189486   2022-05-23
189487   2022-05-24
189488   2022-05-25
189489   2022-05-26
Name: date, Length: 189490, dtype: datetime64[ns]